In [1]:
# Imports
import csv
import pandas as pd
import nltk
import re
import collections
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('rslp')
import matplotlib.pyplot as plt
import heapq
import time
!pip install memory_profiler
%load_ext memory_profiler

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
     |████████████████████████████████| 40kB 5.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/f0/ff/63/fdbff3f1e1b76ad4eae491dd5b190902906b093e93eb86dd5a
Successfully built memory-profiler


In [0]:
# Lendo o CSV
colecao = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_01/master/output/results.csv')
colecao = colecao.astype(str)
documentos = colecao['texto']

# Laboratório 04: Indexação e Processamento de Consulta

# Questão 1

Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

In [0]:
stopwords = stopwords.words("portuguese")
tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
indices = {}
n = 0

for documento in documentos:
    n += 1
    tokens = tokenizador.tokenize(documento.lower())
    for token in set(tokens):
        if (token not in stopwords and len(token) >= 2):
            contador = (n, tokens.count(token))
            if (token not in indices.keys()):
                  indices[token] = []
                  indices[token].append(contador)
            else:
                  indices[token].append(contador)

In [0]:
# Guardando o indice em formato CSV
inverted_index_df = pd.DataFrame.from_dict(indices, orient="index")
inverted_index_df.to_csv('data.csv')
indices = indices

# Questão 2

Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez.


1.   Defina 5 consultas de um termo somente.
2.   Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).
3.   Dê evidências de que sua implementação está correta.
4.   Compare os tempos médios de execução e uso de memória de cada algoritmo.

In [5]:
# Funcao que implementa a consulta documento-por-vez
def documentoPorVez(consulta, indices, k):
    listas_invertidas = []
    r = []
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    for documento in range(1, len(documentos)+1):
        sd = 0
        for lista_invertida in listas_invertidas:
            for i in lista_invertida:
                if (i[0] == documento): 
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, documento))
    return heapq.nlargest(k, r)
 
# Exemplo
documentoPorVez("bolsonaro", indices, 10)  

[(24, 100),
 (18, 70),
 (16, 66),
 (14, 252),
 (11, 179),
 (11, 103),
 (11, 20),
 (10, 16),
 (9, 220),
 (9, 130)]

In [6]:
# Funcao que implementa a consulta termo-por-vez    
def termoPorVez(consulta, indices, k):
    a = {}
    listas_invertidas = []
    r = []
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    for lista_invertida in listas_invertidas:
        for p in lista_invertida:
            d = p[0]
            freq = p[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))
    return heapq.nlargest(k, r)
  
# Exemplo  
termoPorVez("bolsonaro", indices, 10)

[(24, 100),
 (18, 70),
 (16, 66),
 (14, 252),
 (11, 179),
 (11, 103),
 (11, 20),
 (10, 16),
 (9, 220),
 (9, 130)]

In [0]:
 # Definindo consultas
  
consultas = ["presidente", "educação", "privatização", "empresa", "bolsonaro"]

In [0]:
# Executando as consultas 

resultados_documento = []
resultados_termo = []
resultados_documento_tempo = []
resultados_termo_tempo = []
k = 10

for consulta in consultas:
  inicio_documento = time.time()
  resultados_documento.append(documentoPorVez(consulta, indices, k))
  fim_documento = time.time()
  resultados_documento_tempo.append(fim_documento - inicio_documento)
  
  inicio_termo = time.time()
  resultados_termo.append(termoPorVez(consulta, indices, k))
  fim_termo = time.time()
  resultados_termo_tempo.append(fim_termo - inicio_termo)


In [9]:
# Criando e imprimindo a tabela com os dados dos documentos

tabela_consultas = pd.DataFrame()
tabela_consultas['consulta'] = consultas
tabela_consultas['documento_por_vez'] = resultados_documento
tabela_consultas['termo_por_vez'] = resultados_termo
tabela_consultas['comparacao'] = tabela_consultas.documento_por_vez == tabela_consultas.termo_por_vez
tabela_consultas.index += 1
tabela_consultas

,consulta,documento_por_vez,termo_por_vez,comparacao
1,presidente,"[(21, 16), (13, 140), (11, 138), (10, 100), (1...","[(21, 16), (13, 140), (11, 138), (10, 100), (1...",True
2,educação,"[(9, 218), (3, 195), (2, 177), (2, 89), (1, 25...","[(9, 218), (3, 195), (2, 177), (2, 89), (1, 25...",True
3,privatização,"[(1, 245), (1, 221), (1, 204), (1, 184), (1, 62)]","[(1, 245), (1, 221), (1, 204), (1, 184), (1, 62)]",True
4,empresa,"[(10, 177), (9, 107), (6, 262), (4, 95), (3, 2...","[(10, 177), (9, 107), (6, 262), (4, 95), (3, 2...",True
5,bolsonaro,"[(24, 100), (18, 70), (16, 66), (14, 252), (11...","[(24, 100), (18, 70), (16, 66), (14, 252), (11...",True


In [10]:
# Comparando os tempos medios de de execucao

tabela_tempo = pd.DataFrame()
tabela_tempo['tempo_médio_documento_por_vez'] = resultados_documento_tempo
tabela_tempo['tempo_médio_termo_por_vez'] = resultados_termo_tempo
tabela_tempo.index += 1
tabela_tempo

,tempo_médio_documento_por_vez,tempo_médio_termo_por_vez
1,0.002077,0.000160
2,0.000437,0.000031
3,0.000134,0.000007
4,0.000468,0.000034
5,0.001894,0.000192


In [11]:
# Comparando o uso de memória

print('Uso de memória em Documento Por Vez:')
%memit termoPorVez(consultas[0], indices, k)
print(' ')
print('Uso de memória em Termo Por Vez:')
%memit termoPorVez(consultas[0], indices, k)

Uso de memória em Documento Por Vez:
peak memory: 235.73 MiB, increment: 0.08 MiB
 
Uso de memória em Termo Por Vez:
peak memory: 235.73 MiB, increment: 0.00 MiB


# Questão 3

Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).

1.  Defina 5 consultas conjuntivas (AND).
2.  Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).
3.  Dê evidências de que sua implementação está correta.

In [0]:
# Funcao que implementa a consulta conjuntiva (AND)

def consulta_and(consulta, indices, k):
  
  indices_consulta = []
  ranking = []
  
  for palavra in consulta.split(" "):
    if palavra in indices.keys():
        indices_consulta.append(indices[palavra])
  
  todos_os_indices = [item for sublista in indices_consulta for item in sublista]
  todos_os_indices.sort()
  
  for i in range(len(todos_os_indices)):
    score_documento = 0
    d = todos_os_indices.pop()
    cont = 1
    for indice in todos_os_indices:
      if indice[0] == d[0]:
        score_documento += indice[1]
        cont += 1
    if score_documento != 0 and cont == len(indices_consulta):
      score_documento += d[1]
      heapq.heappush(ranking, (score_documento, d[0]))
        
  return heapq.nlargest(k, ranking)

In [0]:
# Definindo consultas

consultas_and = ["presidente", "saúde", "privatização", "direita", "bolsonaro"]

In [15]:
# Criando e imprimindo a tabela com os dados dos documentos

k = 10
resultados_and = []

for consulta in consultas_and:
  score_and = consulta_and(consulta, indices, k)
  resultados_and.append(score_and)


tabela_and = pd.DataFrame()
tabela_and['consulta'] = consultas
tabela_and['AND'] = resultados_and
tabela_and.index += 1
tabela_and

,consulta,AND
1,presidente,[]
2,educação,[]
3,privatização,[]
4,empresa,[]
5,bolsonaro,[]


**Resposta da questão 3.3:** No exemplo a seguir, pode-se perceber a lista invertida de cada termo e visualizar que todos os termos aparecem apenas no primeiro documento. Sendo assim, o algoritmo faz a soma dos scores de cada termo, atuibuindo como score do documento no resultado. 

In [16]:
consulta = "governo quer investir educação"
termos_consulta = consulta.split()
indices_termos = [indices[termo] for termo in termos_consulta]
  
score_and = consulta_and(consulta, indices, 10)
#score_and = (score_and[0][::-1])

termos_consulta.append('')
indices_termos.append('')
a = ['', '', '', '', consulta]
b = ['', '', '', '', score_and]

tabela_q3 = pd.DataFrame()
tabela_q3['termos'] = termos_consulta
tabela_q3['lista_invertida'] = indices_termos
tabela_q3['consulta'] = a
tabela_q3['resultado'] = b
tabela_q3.index += 1
tabela_q3

,termos,lista_invertida,consulta,resultado
1,governo,"[(1, 4), (5, 3), (6, 1), (8, 2), (10, 1), (12,...",,
2,quer,"[(15, 1), (17, 1), (18, 2), (20, 1), (25, 2), ...",,
3,investir,"[(267, 1)]",,
4,educação,"[(4, 1), (7, 1), (15, 1), (22, 1), (58, 1), (6...",,
5,,,governo quer investir educação,[]
